https://arxiv.org/pdf/1905.02244
Depthwise separable convolutions are defined by
two separate layers: light weight depthwise convolution for
spatial filtering and heavier 1x1 pointwise convolutions for
feature generation (setting groups in pytorch)!

use swish nonlinearity: h-swish[x] = x * ReLU6(x + 3) / 6

each layer activation memory typically halves every time the resolution drops. Incidentally, we find that most of the benefits swish are realized by using them only in the deeper layers. 

Bneck block:

expansion layer (1x1 conv) + swish
depthwise conv + swish
squeeze and excitation: global avg pooling, two fully connected layers with swish in between, sigmoid at the end, scale the input to the depthwise conv
projection layer (1x1 conv)

https://medium.com/data-science/a-basic-introduction-to-separable-convolutions-b99ec3102728

Mistakes: Padding should be kernel_size // 2, not hardcoded 3


In [3]:
import torch
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset

class BottleneckBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, reduction_factor, exp_factor, stride, isSwish):
        super().__init__()

        self.isSwish = isSwish
        self.use_skip = (in_channels == out_channels and stride == 1)

        expanded_channels = int(in_channels * exp_factor)

        self.expansion_layer = nn.Sequential(
            nn.Conv2d(in_channels, expanded_channels, kernel_size=1, stride = 1),
            nn.BatchNorm2d(expanded_channels),
        )

        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(expanded_channels, expanded_channels, kernel_size, padding=kernel_size // 2, groups=expanded_channels, stride=stride),
            nn.BatchNorm2d(expanded_channels)            
        )

        self.excitation_layers = nn.Sequential(
            nn.Linear(expanded_channels, expanded_channels // reduction_factor),
            nn.SiLU(),
            nn.Linear(expanded_channels // reduction_factor, expanded_channels),
        )

        self.projection_layer = nn.Sequential(
            nn.Conv2d(expanded_channels, out_channels, kernel_size=1, stride=1),
            nn.BatchNorm2d(out_channels)
        )

    def hard_sigmoid(self, x):
        return nn.functional.relu6(x + 3) / 6

    def swish(self, x):
        return x * nn.functional.sigmoid(x)

    def relu(self, x):
        return nn.functional.relu(x)

    def forward(self, x):
        # Expansion + activation
        features = self.expansion_layer(x)
        features = self.swish(features) if self.isSwish else self.relu(features)
        
        # Depthwise + activation
        features = self.depthwise_conv(features)
        features = self.swish(features) if self.isSwish else self.relu(features)
        
        # SE module
        pooled = self.global_avg_pool(features)
        squeezed = pooled.squeeze(-1).squeeze(-1)
        excitation = self.excitation_layers(squeezed)
        excitation = torch.sigmoid(excitation)  # Use sigmoid, not hard_sigmoid
        excitation = excitation.unsqueeze(-1).unsqueeze(-1)
        features = features * excitation
        
        # Projection (no activation)
        features = self.projection_layer(features)
        
        return features + x if self.use_skip else features

class MobileNetV3(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 16, stride=2, kernel_size=3, padding=1),
            BottleneckBlock(16, 16, 3, 4, 1, 1, False),
            BottleneckBlock(16, 24, 3, 4, 4.5, 2, False),
            BottleneckBlock(24, 24, 3, 4, 3.6, 1, False),

            BottleneckBlock(24, 40, 5, 4, 4, 2, True),
            BottleneckBlock(40, 40, 5, 4, 6, 1, True),
            BottleneckBlock(40, 40, 5, 4, 6, 1, True),

            BottleneckBlock(40, 48, 5, 4, 3, 1, True),

            BottleneckBlock(48, 48, 5, 4, 3, 1, True),
            BottleneckBlock(48, 96, 5, 4, 6, 2, True),

            nn.Conv2d(96, 192, kernel_size=1, stride=1),
            nn.BatchNorm2d(192),
            nn.ReLU(),

            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Conv2d(192, 384, kernel_size=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(384, 384, kernel_size=1, stride=1),
        )

        self.linear = nn.Linear(384, 100)

    def forward(self, x):
        x = self.layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.linear(x)
        return x

In [4]:
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision('high')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    torch.cuda.empty_cache()

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Mild to avoid over-distortion
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4865, 0.4409], std=[0.2673, 0.2564, 0.2761]),
    transforms.RandomErasing(p=0.5)  # Apply after normalization for consistency
])

test_transform = transforms.Compose([
    transforms.ToTensor(), # Moved ToTensor before Normalize (good practice)
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

# Load raw datasets
cifar_train_raw = datasets.CIFAR100(root="./data", train=True, download=True, transform=None)

train_size = int(0.9 * len(cifar_train_raw))  # 48,000

train_indices = list(range(0, train_size))
val_indices = list(range(train_size, len(cifar_train_raw)))

# Create datasets with appropriate transforms
cifar_train = Subset(
    datasets.CIFAR100(root="./data", train=True, transform=train_transform),
    train_indices
)
cifar_val = Subset(
    datasets.CIFAR100(root="./data", train=True, transform=test_transform),
    val_indices
)
# Use original test set (10,000 samples) - close to 10% of 60,000
cifar_test = datasets.CIFAR100(root="./data", train=False, transform=test_transform)

train_loader = DataLoader(
    cifar_train,  # Use directly
    batch_size=1024,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=6
)

val_loader = DataLoader(
    cifar_val,  # Use directly
    batch_size=1024,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=6
)

test_loader = DataLoader(
    cifar_test,
    batch_size=1024,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=6
)

num_classes = 100

model = MobileNetV3().to(device)

num_epochs = 40
loss_function = nn.CrossEntropyLoss(label_smoothing=0.1)
base_lr = 4e-3

batch_scale = 1024 / 256  # 4x larger batches
scaled_lr = base_lr * batch_scale**0.5  # Square root scaling

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=3e-3,  # Keep this for now, let OneCycleLR handle it
    weight_decay=5e-4
)

scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=5e-3,                # Slightly lower peak LR for stability
    epochs=num_epochs,          # Keep 45 epochs
    steps_per_epoch=len(train_loader),
    pct_start=0.4,              # Increase warmup to 40% (18 epochs)
    anneal_strategy='cos',
    div_factor=12.0,            # Start LR = 5e-3 / 12 = 4.2e-4
    final_div_factor=400.0      # Final LR = 5e-3 / 400 = 1.25e-5
)

best_val_loss = float('inf')

for epoch in range(num_epochs):
    print(f'Starting Epoch {epoch+1}')
    model.train()

    current_loss = 0.0
    num_batches = 0

    for i, data in enumerate(train_loader):
        inputs, targets = data
        inputs, targets = inputs.to(device), targets.to(device)
            
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()

        current_loss += loss.item()
        num_batches += 1

        if i % 50 == 0:
            print(f'Batch {i}/{len(train_loader)}, Loss: {loss.item():.4f}')


    avg_train_loss = current_loss / num_batches
    print(f'Epoch {epoch+1} finished')
    print(f'Training - Loss: {avg_train_loss:.4f}')

    if (epoch + 1) % 2 == 0:
        model.eval()
        val_loss = 0.0
        val_batches = 0

        print(f'Epoch {epoch+1} finished')
        print(f'average training loss is {avg_train_loss:.4f}')

        with torch.no_grad():
            for val_data in val_loader:
                val_inputs, val_targets = val_data
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)  # Convert inputs to FP16

                val_outputs = model(val_inputs)
                val_batch_loss = loss_function(val_outputs, val_targets)

                val_loss += val_batch_loss.item()
                val_batches += 1


        avg_val_loss = val_loss / val_batches

        print(f'Epoch {epoch+1} finished')
        print(f'Training - Loss: {avg_train_loss:.4f}')
        print(f'Validation - Loss: {avg_val_loss:.4f}')

if torch.cuda.is_available():
    torch.cuda.empty_cache()


Using device: cuda
GPU Memory: 15.8 GB
Starting Epoch 1
Batch 0/44, Loss: 4.6079
Epoch 1 finished
Training - Loss: 4.5507
Starting Epoch 2
Batch 0/44, Loss: 4.4488
Epoch 2 finished
Training - Loss: 4.3446
Epoch 2 finished
average training loss is 4.3446
Epoch 2 finished
Training - Loss: 4.3446
Validation - Loss: 4.3033
Starting Epoch 3
Batch 0/44, Loss: 4.2328
Epoch 3 finished
Training - Loss: 4.1669
Starting Epoch 4
Batch 0/44, Loss: 4.0571
Epoch 4 finished
Training - Loss: 4.0144
Epoch 4 finished
average training loss is 4.0144
Epoch 4 finished
Training - Loss: 4.0144
Validation - Loss: 3.9141
Starting Epoch 5
Batch 0/44, Loss: 3.9380
Epoch 5 finished
Training - Loss: 3.8854
Starting Epoch 6
Batch 0/44, Loss: 3.7763
Epoch 6 finished
Training - Loss: 3.7690
Epoch 6 finished
average training loss is 3.7690
Epoch 6 finished
Training - Loss: 3.7690
Validation - Loss: 3.6820
Starting Epoch 7
Batch 0/44, Loss: 3.7468
Epoch 7 finished
Training - Loss: 3.6703
Starting Epoch 8
Batch 0/44, Los

In [5]:
def evaluate_test_set(model):
    model.eval()
    correct = 0
    total = 0
    
    print("Starting evaluation...")
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            
            # Use autocast for consistency if you trained with it
            outputs = model(images)
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the network on the test images: {accuracy:.2f}%')
    return accuracy

print("\n=== Running standard evaluation ===")
standard_accuracy = evaluate_test_set(model)   
print(f'Standard Test Accuracy: {standard_accuracy:.4f} ({standard_accuracy*100:.2f}%)')


=== Running standard evaluation ===
Starting evaluation...
Accuracy of the network on the test images: 48.30%
Standard Test Accuracy: 48.3000 (4830.00%)
